In [3]:
##Libraries
import sys
import math

# #print(len(sys.argv))
# if len(sys.argv) == 2:
#     infile = sys.argv[1]
# else:
#     infile = "data/point100.lst"
infile = "data/point100.lst"

print(infile)

data/point100.lst


In [4]:
#loading data
def readpoints(infile):
    point_list = []
    n = 0
    with open(infile) as file:
        for line in file:
            n += 1
            line = line.strip().split()
            #files with index column starting with "point"
            if line[0].lower().startswith("point"):
                point_list.append((line[0].replace("p", "P", 1), *[float(value) for value in line[1:]])) #unpacking opperator
            #for files with no index column
            else:
                point_list.append((f"Point{1+n}",*[float(value) for value in line])) #unpacking opperator       
    return(point_list)

In [5]:
point_data = readpoints(infile)
print(point_data)

[('Point00', 0.2605, 0.6913, 0.2874, 0.4148), ('Point01', 0.7535, 0.1888, 0.6069, 0.4798), ('Point02', 0.6464, 0.4533, 0.6959, 0.9934), ('Point03', 0.8025, 0.3499, 0.9197, 0.7967), ('Point04', 0.5298, 0.2574, 0.0364, 0.978), ('Point05', 0.3617, 0.8717, 0.0083, 0.8315), ('Point06', 0.8064, 0.886, 0.6524, 0.3354), ('Point07', 0.1975, 0.3999, 0.7562, 0.65), ('Point08', 0.9166, 0.7109, 0.7922, 0.416), ('Point09', 0.521, 0.4179, 0.6319, 0.3168), ('Point10', 0.2597, 0.1704, 0.1205, 0.3321), ('Point11', 0.2301, 0.5692, 0.1636, 0.8524), ('Point12', 0.9612, 0.6027, 0.1892, 0.7527), ('Point13', 0.5605, 0.0207, 0.7787, 0.1729), ('Point14', 0.078, 0.4023, 0.6483, 0.0607), ('Point15', 0.3805, 0.5994, 0.4684, 0.0572), ('Point16', 0.3477, 0.2021, 0.6284, 0.5274), ('Point17', 0.4602, 0.6944, 0.595, 0.0421), ('Point18', 0.0167, 0.7986, 0.0388, 0.4048), ('Point19', 0.0517, 0.94, 0.1744, 0.0285), ('Point20', 0.1104, 0.8377, 0.4869, 0.6085), ('Point21', 0.0271, 0.5167, 0.1315, 0.294), ('Point22', 0.7541, 

In [6]:
def euclidean_dist(pointA, pointB):
    """Takes two points as input and calculates the euclidean distance between them"""

    coords1 = pointA[1:]
    coords2 = pointB[1:]
    
    # Verify both points have the same number of dimensions
    if len(coords1) != len(coords2):
        raise ValueError("Points must have the same number of dimensions")
    
    # Start a square sum
    square_sum = 0

    # Loop through the point values, and measure the square sum of the differences
    for i in range(len(coords1)):
        square_sum += (coords2[i] - coords1[i]) ** 2

    # Get the euclidean distance
    distance = math.sqrt(square_sum)
    return distance

# Calculate the distance between each point
distance_matrix = {}
for i in range(len(point_data)):
    for j in range(i + 1, len(point_data)):
        dist = euclidean_dist(point_data[i], point_data[j])
        print(dist)
        distance_matrix[(i, j)] = dist

print(distance_matrix)
print(len(distance_matrix))

0.7757966872834661
0.840959582857583
0.9777366005218379
0.800616599877869
0.5425138707904158
0.6895246623580624
0.6033037709147855
0.8280554631182623
0.5204729195645054
0.5532019070827576
0.4718609646919313
0.7890818081796083
0.9162945268853241
0.6102973619474362
0.4283507558065002
0.6130907273805404
0.5228888505217911
0.36449100126066214
0.5171516412040089
0.34825695972945037
0.3519348945472727
0.7747235119705611
0.4560065788999102
0.9095796281799632
1.0233168766320626
0.3955898633686156
0.9649863522350977
0.8561233731186177
0.7485508265976333
0.4869138835564252
0.8917433599416371
0.8443520474304542
0.6892965617207154
0.961017559673079
0.47031620214489744
0.333731793510897
0.579789470411459
0.352887871709981
0.5714024063652515
0.7648058250301183
0.727947312653876
0.6318610131983141
1.1829572477482015
0.18057776718079102
0.8722707664481254
0.7545998210972488
0.6586542492081866
0.44342699511870043
0.4803619989133195
0.6058595629351741
0.46058942671320635
0.8582464156639399
0.65679962697

In [7]:
# Calculate the distance between the two points that are ruthest away from each other
max_distance = max(distance_matrix.values())
print(max_distance)

# Measure QT as 30% of the maximum distance
quality_threshold = 0.3 * max_distance
print(f"Quality Threshold (30% of diameter): {quality_threshold}")

1.66043533147184
Quality Threshold (30% of diameter): 0.498130599441552


In [8]:
def candidate_cluster(center_idx, point_data, distance_matrix, threshold, used_set):
    """
    Forms a candidate cluster starting from center_idx.
    Only includes points that are not in used_set.
    Keeps adding points while the cluster's diameter remains within threshold.
    """

    # Start with the center point
    cluster = [center_idx]

    # Make a list of potential neighbours with their distance to the center point
    distance_to_center = []
    for i in range(len(point_data)):
        # Try each point that is not the center point or an already used point in another cluster
        if i != center_idx and i not in used_set:
            ######used_set.add(point_data[i])
            # Get the distance from centre_idx to i
            key = (center_idx, i) if center_idx < i else (i, center_idx)
            ##print(key)
            dist = distance_matrix[key]
            distance_to_center.append((i, dist))
    
    # Sort potential neighbor by distance (closest first)
    distance_to_center.sort(key = lambda x: x[1])
    # print("distance_to_center")
    # print(distance_to_center)

    # Try adding each neighbor
    for idx, _ in distance_to_center:
        # Tentatively add the point
        trial_cluster = cluster + [idx]
        
        # print("trial_cluster_len")
        # print(trial_cluster)
        # print(len(trial_cluster))

        # Check all pairwise distances in trial_cluster
        max_dist = 0
        for i in range(len(trial_cluster)):
            for j in range(i + 1, len(trial_cluster)):
                a, b = trial_cluster[i], trial_cluster[j]
                key = (a,b) if a < b else (b, a)
                ##print(key)
                d = distance_matrix[key]
                if d > max_dist:
                    max_dist = d
                    # print("max_dist")
                    # print(max_dist)

        # If the max distance is within the threshold, accept the new point
        if max_dist <= threshold:
            cluster.append(idx)
        #If not, skip it

    return cluster


In [9]:
used_set = set()
for candidate_center in range(len(point_data)):
    cluster = candidate_cluster(candidate_center, point_data, distance_matrix, quality_threshold, used_set)
    used_set.add(i)
    #print(used_set)
    print(cluster)

[0, 65, 75, 44, 36, 78, 21, 18]
[1, 83, 33, 92, 28]
[2, 86, 45, 60, 27, 39, 28]
[3, 67, 28, 45, 27, 86, 25, 43]
[4, 24, 46, 53, 37, 66]
[5, 30, 38, 80, 63]
[6, 70, 61, 74, 8, 35, 47]
[7, 96, 16, 79, 95]
[8, 61, 6, 93, 70, 35]
[9, 73, 93, 83, 1, 23]
[10, 72, 59, 36, 21, 75]
[11, 30, 98, 57, 56]
[12, 32, 69, 66, 63]
[13, 71, 91, 9, 16]
[14, 89, 51, 15, 81, 17]
[15, 17, 65, 51, 89, 81]
[16, 96, 41, 7, 95]
[17, 15, 47, 74, 35, 73]
[18, 78, 21, 36, 44, 0, 65, 75]
[19, 78, 18, 44, 65, 36]
[20, 26, 49, 54, 55, 0]
[21, 36, 75, 18, 65, 0, 78, 44]
[22, 69, 32, 80, 63, 12]
[23, 37, 33, 53, 29, 46, 83, 66, 92]
[24, 46, 66, 53, 4, 37]
[25, 43, 64, 28, 3, 62, 67]
[26, 20, 54, 49, 0, 55]
[27, 79, 45, 3, 86, 2, 67, 28]
[28, 3, 86, 25, 45, 67, 27, 43]
[29, 12, 23, 66, 46, 31, 37]
[30, 11, 5, 80]
[31, 66, 60, 86, 2]
[32, 22, 69, 12, 63, 80]
[33, 37, 83, 1, 92, 53, 23, 40, 29]
[34, 93, 62, 28, 1, 3]
[35, 73, 74, 47, 6, 70, 8]
[36, 21, 75, 65, 18, 0, 78, 44]
[37, 33, 53, 23, 46, 66, 83, 29, 92]
[38, 44, 7

In [ ]:
def best_cluster(point_data, distance_matrix, threshold, used_set):
    """
    Tries to form a cluster from each unused point.
    Returns the best cluster found: the one with the most points (and tightest if tied).
    """

    best_cluster = []
    best_diameter = float("inf")    

    # Try to form a candidate cluster from every unused point
    for i in range(len(point_data)):
        #print(i)
        if i in used_set:
            continue # Skip if point has already been used
        ##print(used_set)
        # Form a candidate cluster starting from point i
        cluster = candidate_cluster(i, point_data, distance_matrix, threshold, used_set)

        # Only consider clusters that have more than 1 point
        if len(cluster) < 2:
            continue

        # Compute the diameter of this cluster 
        max_dist = 0
        for a in range(len(cluster)):
            for b in range(a + 1, len(cluster)):
                p1, p2 = cluster[a], cluster[b]
                key = (p1, p2) if p1 < p2 else (p2, p1)
                d = distance_matrix[key]
                if d > max_dist:
                    max_dist = d # Track the largest distance in the cluster

        # Compare this cluster with the current best one
        if (len(cluster) > len(best_cluster)) or (len(cluster) == len(best_cluster) and max_dist < best_diameter):
            best_cluster = cluster
            best_diameter = max_dist
        
    return best_cluster

In [10]:
used_set_2 = set()
for i in range(len(point_data)):
    best = best_cluster(point_data, distance_matrix, quality_threshold, used_set_2)
    if type(best) != None:
        used_set_2.update(best) 
    #print(used_set)
    print(f"{i} best cluster")
    print(best)

0 best cluster
[92, 1, 83, 33, 77, 40, 37, 29, 53]
1 best cluster
[0, 65, 75, 44, 36, 78, 21, 18]
2 best cluster
[3, 67, 28, 45, 27, 86, 25, 43]
3 best cluster
[6, 70, 61, 74, 8, 35, 47]
4 best cluster
[22, 69, 32, 80, 63, 12]
5 best cluster
[14, 89, 51, 15, 81, 17]
6 best cluster
[50, 55, 98, 49, 20, 39]
7 best cluster
[7, 96, 16, 79, 95]
8 best cluster
[82, 91, 10, 72, 42]
9 best cluster
[30, 11, 5, 57]
10 best cluster
[41, 59, 56, 52]
11 best cluster
[76, 99, 97, 60]
12 best cluster
[46, 66, 24, 31]
13 best cluster
[9, 73, 93, 23]
14 best cluster
[13, 71, 58, 85]
15 best cluster
[68, 94]
16 best cluster
[26, 54]
17 best cluster
[62, 64]
18 best cluster
[48, 84]
19 best cluster
[]
20 best cluster
[]
21 best cluster
[]
22 best cluster
[]
23 best cluster
[]
24 best cluster
[]
25 best cluster
[]
26 best cluster
[]
27 best cluster
[]
28 best cluster
[]
29 best cluster
[]
30 best cluster
[]
31 best cluster
[]
32 best cluster
[]
33 best cluster
[]
34 best cluster
[]
35 best cluster
[]
36 b